In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [61]:
df = pd.read_csv('speed_dating_data.csv', encoding = "ISO-8859-1")

In [9]:
data1 = pd.concat([#df.iloc[:,0], #ID
                  df.iloc[:,2], #gender
                  df.iloc[:,9:10], #order of date
                  df.iloc[:, 11:12], #partner's id
                  df.iloc[:, 12:13], #match, 
                  df.iloc[:,13:17], #int_corr, samerace, age of partner, race of partner 
                  df.iloc[:,17:23], #stated preferences
                  df.iloc[:,23:24], #decision of partner - dec-o
                  df.iloc[:,24:32],   # rating by partner for 6 attributes, probability of matching with partner #amb-o has lots of missing values - try eliminating
                  #df.iloc[:,32:35], # met, age, field
                  df.iloc[:,39:42],#race, importance of race, importance of religion
                  #df.iloc[:,42:43], #from   (DROPPED ZIPCODE AND INCOME AS THEY HAVE TOO MANY MISSING VALUES)
                  df.iloc[:,45:48], #goal, date, go_out
                  #df.iloc[:,48:50], #career, career_coded
                  df.iloc[:,50:67], #interests
                  #df.iloc[:,67:69], #exphappy, expnum (DROPPED - TOO MANY MISSING VALUES)
                  df.iloc[:,69:74],  #what subject looks for in the opposite sex - 6 attributes
                  df.iloc[:,74:75],  #shar1_1
                  df.iloc[:,81:87], #attr2_1,..shar2_1
                  df.iloc[:,87:92],  #attr3_1.. #91 is amb3_1 (how do you measure up)
                  df.iloc[:,97:98], #decision of subject - dec
                  df.iloc[:,98:102], #ratings by subject for partner for the 6 attributes. 101 is fun
                  df.iloc[:,102:104], #amb,shar
                  df.iloc[:,104:107]],axis=1) #like, prob, met

data1 = data1.dropna()
print(np.shape(data1))
                

(5561, 73)


### <font color='red'> multiple linear regression assumes attributes to be independent, but some of ours aren't (such as self-evaluation and evaluation by others)</font>

In [11]:
#Baseline regression model (without any feature selection -  contains 67 predictors)
#these 67 features account for 65 % of the variance in the dataset for y = dec
#TODO : replace dec with match and dec_o 
Y_reg = data1.dec
X_reg1 = data1[['gender','order','int_corr','samerace','age_o','race_o', 'pf_o_att','pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb','pf_o_sha','attr_o','sinc_o','intel_o','fun_o','amb_o','shar_o','like_o','prob_o','race','imprace','imprelig','goal','date','go_out','sports','tvsports','exercise','dining','museums','art','hiking','gaming','clubbing','reading','tv','theater','movies','concerts','music','shopping','yoga','attr1_1','sinc1_1','intel1_1','fun1_1','amb1_1','shar1_1','attr2_1','sinc2_1','intel2_1','fun2_1','amb2_1','shar2_1','attr3_1','sinc3_1','fun3_1','intel3_1','amb3_1','attr','sinc','intel','fun','amb','shar','prob']]
model = sm.OLS(Y_reg, X_reg1)
result_reg = model.fit()
result_reg.summary()

#results : p-values imply only few variables are statistically significant. F-statistic value is also less.


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    dec   R-squared:                       0.635
Model:                            OLS   Adj. R-squared:                  0.631
Method:                 Least Squares   F-statistic:                     142.9
Date:                Wed, 11 Dec 2019   Prob (F-statistic):               0.00
Time:                        00:49:13   Log-Likelihood:                -2788.8
No. Observations:                5561   AIC:                             5712.
Df Residuals:                    5494   BIC:                             6155.
Df Model:                          67                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
gender         0.0620      0.018      3.391      0.001       0.026       0.098
order          0.0009      0.001      0.868      0.386      -0.001       0.003
int_corr       0.0001      0.020      0.005      0.996      -0.038       0.039
samerace       0.0060      0.012      0.494      0.622      -0.018       0.030
age_o         -0.0021      0.002     -1.277      0.202      -0.005       0.001
race_o        -0.0026      0.005     -0.558      0.577      -0.012       0.006
pf_o_att      -0.0001      0.002     -0.062      0.950      -0.004       0.004
pf_o_sin      -0.0024      0.002     -1.061      0.289      -0.007       0.002
pf_o_int      -0.0007      0.002     -0.312      0.755      -0.005       0.004
pf_o_fun      -0.0016      0.002     -0.673      0.501      -0.006       0.003
pf_o_amb      -0.0024      0.002     -1.088      0.277      -0.007       0.002
pf_o_sha      -0.0028      0.002     -1.195      0.232      -0.007       0.002
attr_o        -0.0175      0.004     -4.355      0.000      -0.025      -0.010
sinc_o         0.0100      0.004      2.247      0.025       0.001       0.019
intel_o        0.0035      0.006      0.641      0.521      -0.007       0.014
fun_o         -0.0026      0.004     -0.596      0.551      -0.011       0.006
amb_o          0.0043      0.004      1.046      0.296      -0.004       0.012
shar_o        -0.0065      0.004     -1.744      0.081      -0.014       0.001
like_o        -0.0002      0.005     -0.043      0.965      -0.010       0.010
prob_o         0.0110      0.003      3.552      0.000       0.005       0.017
race           0.0102      0.005      2.051      0.040       0.000       0.020
imprace       -0.0057      0.002     -2.431      0.015      -0.010      -0.001
imprelig      -0.0057      0.002     -2.426      0.015      -0.010      -0.001
goal          -0.0075      0.004     -1.794      0.073      -0.016       0.001
date          -0.0068      0.004     -1.575      0.115      -0.015       0.002
go_out         0.0076      0.006      1.348      0.178      -0.003       0.019
sports        -0.0086      0.003     -2.919      0.004      -0.014      -0.003
tvsports       0.0014      0.003      0.545      0.586      -0.004       0.007
exercise      -0.0066      0.003     -2.479      0.013      -0.012      -0.001
dining        -0.0058      0.004     -1.458      0.145      -0.014       0.002
museums       -0.0132      0.006     -2.292      0.022      -0.024      -0.002
art            0.0104      0.005      2.058      0.040       0.000       0.020
hiking        -0.0036      0.002     -1.471      0.141      -0.008       0.001
gaming         0.0071      0.003      2.815      0.005       0.002       0.012
clubbing       0.0050      0.002      2.114      0.035       0.000       0.010
reading        0.0037      0.003      1.202      0.230      -0.002       0.010
tv

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

#Recursive feature elimination - sending a subset of previous features
y = data1.dec
X = data1[['gender','attr_o','sinc_o','intel_o','fun_o','amb_o','shar_o','prob_o','race','imprace','imprelig','goal','attr1_1','sinc1_1','intel1_1','fun1_1','amb1_1','shar1_1','attr2_1','sinc2_1','intel2_1','fun2_1','amb2_1','shar2_1','attr3_1','sinc3_1','fun3_1','intel3_1','amb3_1','attr','sinc','intel','fun','amb','shar','prob']]
estimator = SVR(kernel="linear")
selector = RFE(estimator, 20, step=1)
selector = selector.fit(X, y)
selector.ranking_


In [ ]:
selector.support_
#method selects following 20 best features from the above subset of features
#gender, attr_o, sinc_o, intel_o, shar_o, prob_o, race, imprace, goal, attr3_1, sinc3_1, fun3_1, intel3_1, amb3_1, attr, sinc, fun, amb, shar, prob

In [ ]:
#using the above features after feature selection, regression model gives better values. p-values < 0.0001 - most of these features are statistically significant
#Use these features in classification 

#TODO: feature selection with y = match

y = data1.dec
X_reg4 = data1[['gender','attr_o','sinc_o','shar_o','prob_o','race','imprace','goal','attr3_1','sinc3_1','fun3_1','intel3_1','amb3_1','attr','sinc','fun','amb','shar','prob']]

model = sm.OLS(y, X_reg4)
result_reg = model.fit()
result_reg.summary()

# Feature Engineering

In [75]:
df2 = pd.concat([
    df.iid, df.partner, df.pid, # id of self and partner, to be removed later
    df.gender,
    df.age, df.age_o,
    df.int_corr, # correlation of interests
    df.samerace,
    df.goal,
    df.date,
    df.exphappy, # expected happiness with people you will meet
    df.loc[:, 'attr3_1':'amb3_1'],  # self eval, assuming to be accurate
    df.loc[:, 'attr1_1':'shar1_1'],  # what's important to you, add to 100  
    df.dec_o # decision of partner
], axis=1)

## Fill Missing Values

### pid (partner's unique ID)

In [76]:
df2[df2.pid.isnull()]  # The missing pid comes from same person in one night

,iid,partner,pid,gender,age,age_o,int_corr,samerace,goal,date,...,fun3_1,intel3_1,amb3_1,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,dec_o
1755,122,7,NaN,1,22.0,NaN,-0.12,0,1.0,3.0,...,8.0,8.0,8.0,40.0,15.0,10.0,20.0,5.0,10.0,0
1765,123,7,NaN,1,18.0,NaN,-0.29,0,2.0,5.0,...,7.0,7.0,8.0,50.0,10.0,20.0,5.0,5.0,10.0,0
1775,124,7,NaN,1,22.0,NaN,-0.05,0,2.0,4.0,...,10.0,10.0,10.0,40.0,10.0,10.0,10.0,10.0,20.0,0
1785,125,7,NaN,1,21.0,NaN,0.15,0,1.0,3.0,...,10.0,9.0,10.0,20.0,15.0,20.0,15.0,12.0,18.0,0
1795,126,7,NaN,1,22.0,NaN,0.01,0,4.0,6.0,...,9.0,9.0,9.0,25.0,40.0,15.0,10.0,5.0,5.0,0
1805,127,7,NaN,1,22.0,NaN,0.38,0,6.0,5.0,...,9.0,9.0,8.0,25.0,25.0,25.0,15.0,5.0,5.0,0
1815,128,7,NaN,1,21.0,NaN,-0.05,0,1.0,3.0,...,7.0,8.0,7.0,40.0,15.0,10.0,20.0,10.0,5.0,0
1825,129,7,NaN,1,NaN,NaN,0.09,0,3.0,3.0,...,8.0,8.0,8.0,40.0,20.0,20.0,20.0,NaN,NaN,0
1835,130,7,NaN,1,20.0,NaN,-0.40,0,1.0,6.0,...,6.0,6.0,6.0,80.0,10.0,10.0,NaN,NaN,NaN,0
1845,131,7,NaN,1,19.0,NaN,-0.14,0,2.0,7.0,...,4.0,6.0,6.0,25.0,10.0,20.0,20.0,10.0,15.0,0


In [88]:
df2.pid.fillna(df2.pid.max() + 1, inplace=True)  # Assign a new pid to the person missing
assert df2.pid.isna().sum() == 0

### age

In [78]:
df2[df2.age.isnull()] 

,iid,partner,pid,gender,age,age_o,int_corr,samerace,goal,date,...,fun3_1,intel3_1,amb3_1,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,dec_o
828,58,1,66.0,0,NaN,29.0,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
829,58,2,67.0,0,NaN,22.0,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
830,58,3,68.0,0,NaN,27.0,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
831,58,4,69.0,0,NaN,28.0,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
832,58,5,70.0,0,NaN,26.0,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7493,512,18,548.0,0,NaN,30.0,-0.19,1,1.0,3.0,...,8.0,7.0,7.0,20.0,20.0,20.0,20.0,5.0,15.0,0
7494,512,19,549.0,0,NaN,28.0,0.26,1,1.0,3.0,...,8.0,7.0,7.0,20.0,20.0,20.0,20.0,5.0,15.0,1
7495,512,20,550.0,0,NaN,30.0,0.63,1,1.0,3.0,...,8.0,7.0,7.0,20.0,20.0,20.0,20.0,5.0,15.0,1
7496,512,21,551.0,0,NaN,27.0,0.37,0,1.0,3.0,...,8.0,7.0,7.0,20.0,20.0,20.0,20.0,5.0,15.0,0


In [83]:
df2['age'] = df2.groupby('gender').transform(lambda group: group.fillna(group.median()))
df2['age_o'] = df2.groupby('gender').transform(lambda group: group.fillna(group.median()))

In [87]:
assert df2.age.isna().sum() == 0
assert df2.age_o.isna().sum() == 0

# Baseline Logistic Regression
No feature engineering;<br>
Treat ordinal variables as continuous, drop all other categorical variables<br>
Remove attributes that might be linearly dependent on others<br>

In [17]:
df2 = pd.concat([
    df.age,
    df.int_corr, # correlation of interests
    df.samerace,
    df.goal,
    df.date,
    df.exphappy, # expected happiness with people you will meet
    df.loc[:, 'attr3_1':'amb3_1'],  # self eval, assuming to be accurate
    df.dec_o # decision of partner
], axis=1)

In [18]:
df2.isna().sum()

age          95
int_corr    158
samerace      0
goal         79
date         97
exphappy    101
attr3_1     105
sinc3_1     105
fun3_1      105
intel3_1    105
amb3_1      105
dec_o         0
dtype: int64

In [13]:
X = df2.iloc[:,:-1]
y = df2.dec_o

In [14]:
lr = LogisticRegression()
lr.fit(X,y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [7]:
df3 = pd.concat([df.id, df.age, df.age_o], axis=1)
df3['age_diff'] = df3.age -df3.age_o
df3

,id,age,age_o,age_diff
0,1.0,21.0,27.0,-6.0
1,1.0,21.0,22.0,-1.0
2,1.0,21.0,22.0,-1.0
3,1.0,21.0,23.0,-2.0
4,1.0,21.0,24.0,-3.0
...,...,...,...,...
8373,22.0,25.0,26.0,-1.0
8374,22.0,25.0,24.0,1.0
8375,22.0,25.0,29.0,-4.0
8376,22.0,25.0,22.0,3.0
